# Intialize

In [1]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from torchvision import models

import matplotlib.pyplot as plt
import numpy as np
import torch
import cv2
import os

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data Load

In [3]:
DATA_PATH = './data/'
DATA_PATH_LIST = os.listdir(DATA_PATH)
DATA_PATH_LIST = [DATA_PATH + path for path in DATA_PATH_LIST]

print(f"Data path list: {DATA_PATH_LIST}")
image_paths = []
for path in tqdm(DATA_PATH_LIST):
    for file in os.listdir(path):
        image_paths.append(path + '/' + file)

print(f"Image paths: {image_paths}")
images = []
for path in tqdm(image_paths):
    image = cv2.imread(path)
    images.append(image)

print(f"Converting to numpy array...")
images = torch.Tensor(images).float()
images = images.permute(0, 3, 1, 2)
print(f"Iamges shape: {images.shape}")

Data path list: ['./data/016-1-1-01-Z109_C', './data/031-1-1-01-Z72_C', './data/010-1-1-01-Z17_C', './data/005-1-1-01-Z17_C', './data/012-1-1-01-Z109_C', './data/008-1-1-01-Z109_C', './data/017-1-1-01-Z72_C', './data/011-1-1-01-Z109_C', './data/019-1-1-01-Z72_C', './data/009-1-1-01-Z17_C', './data/004-1-1-01-Z17_C', './data/006-1-1-01-Z109_C', './data/018-1-1-01-Z109_C', './data/028-1-1-01-Z72_C', './data/007-1-1-01-Z17_C', './data/010-1-1-01-Z109_C', './data/017-1-1-01-Z109_C', './data/014-1-1-01-Z109_C', './data/007-1-1-01-Z109_C', './data/032-1-1-01-Z72_C', './data/006-1-1-01-Z17_C', './data/002-1-1-01-Z17_C', './data/015-1-1-01-Z109_C', './data/029-1-1-01-Z72_C', './data/015-1-1-01-Z72_C', './data/030-1-1-01-Z72_C', './data/008-1-1-01-Z17_C', './data/013-1-1-01-Z109_C', './data/001-1-1-01-Z17_C', './data/003-1-1-01-Z17_C', './data/009-1-1-01-Z109_C', './data/016-1-1-01-Z72_C', './data/020-1-1-01-Z109_C', './data/019-1-1-01-Z109_C', './data/011-1-1-01-Z17_C', './data/018-1-1-01-Z72_

100%|██████████| 37/37 [00:00<00:00, 20267.63it/s]


Image paths: ['./data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000027.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000029.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000004.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000023.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000006.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000019.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000028.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000008.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000005.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000003.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000032.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000014.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000031.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000020.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000021.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000016.jpg', './data/016-1-1-01-Z109_C/016-1-1-01-Z109_C-0000025.jpg', 

100%|█████████▉| 1165/1167 [00:23<00:00, 51.46it/s]

In [ ]:
skeleton_extractor = models.detection.keypointrcnn_resnet50_fpn(pretrained=True).to(device).eval()
bounding_boxer = models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(device).eval()

/home/apple/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/apple/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=KeypointRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=KeypointRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/apple/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights

In [ ]:
# Image -> Bounding Box -> Skeleton -> Skeleton point as list (Vector)
skeleton_list = []
for image in tqdm(images):
    with torch.no_grad():
        prediction = bounding_boxer([image.to(device)])
    prediction = prediction[0]
    x1, y1, x2, y2 = prediction['boxes'][0]

    cropped_image = image[:, int(y1):int(y2), int(x1):int(x2)]

    if cropped_image.shape[1] == 0 or cropped_image.shape[2] == 0:
        skeleton_list.append(np.zeros((17, 2)))
        continue

    with torch.no_grad():
        prediction = skeleton_extractor([cropped_image.to(device)])
    prediction = prediction[0]
    print(f"Prediction: {prediction}")

    skeleton_point = []
    for i in range(17):
        skeleton_point.append(prediction['keypoints'][0][i][0:2].cpu().numpy())
    skeleton_list.append(skeleton_point)

skeleton_points = np.array(skeleton_list)
print(f"Skeleton points shape: {skeleton_points.shape}")

  0%|          | 0/1167 [00:00<?, ?it/s]

  0%|          | 0/1167 [00:00<?, ?it/s]

Prediction: {'boxes': tensor([], device='cuda:0', size=(0, 4)), 'labels': tensor([], device='cuda:0', dtype=torch.int64), 'scores': tensor([], device='cuda:0'), 'keypoints': tensor([], device='cuda:0', size=(0, 17, 3)), 'keypoints_scores': tensor([], device='cuda:0', size=(0, 17))}


IndexError: index 0 is out of bounds for dimension 0 with size 0

In [ ]:
X_train, X_test, y_train, y_test, = train_test_split(images, images, test_size=0.2, random_state=42)
print(f"""X_train shape: {X_train.shape}
y_train shape: {y_train.shape}
X_test shape: {X_test.shape}
y_test shape: {y_test.shape}""")

X_train shape: (1215, 1080, 1920, 3)
y_train shape: (1215, 1080, 1920, 3)
X_test shape: (304, 1080, 1920, 3)
y_test shape: (304, 1080, 1920, 3)


## Model

In [ ]:
import torch.optim as optim
import torch.nn as nn
import torch

Device: cuda:0
